 # <b> jetRacer autonomous driving </b> 

### by lane and stop sign detection- Jonathan Riggs Doron Shaul project

Creating a <b>nvidiaRacecar</b> and a <b>CSI camera</b> objects.

In [1]:
import cv2 as cv
import numpy as np
from time import gmtime, strftime, sleep, time
import logging
import matplotlib.pyplot as plt
from jetcam.csi_camera import CSICamera
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

camera = CSICamera(width=224, height=224)

In [2]:
def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    `initial_img` should be the image before any processing.
    The result image is computed as follows:
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv.addWeighted(initial_img, α, img, β, λ)

This method detects lines (that represents lanes) in the image. The lines have to be at least 10 pixels long.

In [3]:
def detect_line_segments(cropped_edges):
    # tuning min_threshold, minLineLength, maxLineGap is a trial and error process by hand
    rho = 1  # distance precision in pixel, i.e. 1 pixel
    angle = np.pi / 180  # angular precision in radian, i.e. 1 degree
    min_threshold = 10  # minimal of votes
    line_segments = cv.HoughLinesP(cropped_edges, rho, angle, min_threshold, 
                                    np.array([]), minLineLength=10, maxLineGap=15)

    return line_segments

This method gets a frame (in our case from the CSI Camera) and a line, makes the line's length a quarter of the frame, and returns it.

In [4]:
def make_points(frame, line):
    height, width, _ = frame.shape
    slope, intercept = line
    y1 = height  # bottom of the frame
    y2 = int(y1 * 1 / 4)  # make points from quarter of the frame down

    # bound the coordinates within the frame
    x1 = max(-width, min(2 * width, int((y1 - intercept) / slope)))
    x2 = max(-width, min(2 * width, int((y2 - intercept) / slope)))
    return [[x1, y1, x2, y2]]

This method gets a frame and lines. for each line, it checks if the <b>slope</b> is negative or positive. If it's positive, then the line is being added to the <b>right lanes </b>  array, if it's negative, the line is being added to the <b>left lanes </b>array.

In [5]:
def average_slope_intercept(frame, line_segments):
    """
    This function combines line segments into one or two lane lines
    If all line slopes are < 0: then we only have detected left lane
    If all line slopes are > 0: then we only have detected right lane
    """
    lane_lines = []
    if line_segments is None:
        logging.info('No line_segment segments detected')
        return lane_lines

    height, width, _ = frame.shape
    left_fit = []
    right_fit = []

    boundary = 1/2
    left_region_boundary = width * (1 - boundary)  # left lane line segment should be on left 1/2 of the screen
    right_region_boundary = width * boundary # right lane line segment should be on right 1/2 of the screen

    for line_segment in line_segments:
        for x1, y1, x2, y2 in line_segment:
            if x1 == x2:
                logging.info('skipping vertical line segment (slope=inf): %s' % line_segment)
                continue
            fit = np.polyfit((x1, x2), (y1, y2), 1)
            slope = fit[0]
            intercept = fit[1]
            if slope < -0.25 or slope > 0.25:
                if slope < 0:
                    if x1 < left_region_boundary and x2 < left_region_boundary:
                        left_fit.append((slope, intercept))
                else:
                    if x1 > right_region_boundary and x2 > right_region_boundary:
                        right_fit.append((slope, intercept))

    left_fit_average = np.average(left_fit, axis=0)
    if len(left_fit) > 0:
        lane_lines.append(make_points(frame, left_fit_average))

    right_fit_average = np.average(right_fit, axis=0)
    if len(right_fit) > 0:
        lane_lines.append(make_points(frame, right_fit_average))

    logging.debug('lane lines: %s' % lane_lines)

    return lane_lines

This method gets as input a frame (from the camera), lines array, color and width. For each line in the array, this line in <b> being painted </b> on the frame and marks down the lane with the given color and width of the line.

In [6]:
def display_lines(frame, lines, line_color=(0, 255, 255), line_width=5):
    line_image = np.zeros_like(frame)
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv.line(line_image, (x1, y1), (x2, y2), line_color, line_width)
    line_image = cv.addWeighted(frame, 0.8, line_image, 1, 1)
    return line_image

This method gets edges and returns an <b> cropped image </b> that is 1/3 bottom of the original image.
The 1/3 bottom of the frame is our region of intrest in which all the calculations are being preformed.

In [7]:
def region_of_interest(edges):
    height, width = edges.shape
    mask = np.zeros_like(edges)

    # only focus bottom 1/3 of the screen
    polygon = np.array([[
        (0, height * 1 / 3),
        (width, height * 1 / 3),
        (width, height),
        (0, height),
    ]], np.int32)

    cv.fillPoly(mask, polygon, 255)
    cropped_edges = cv.bitwise_and(edges, mask)
    return cropped_edges

In [8]:
# define a range of black color in HSV

lower_black = np.array([0, 0, 0])
upper_black = np.array([227, 100, 70])


# Rectangular Kernel
rectKernel = cv.getStructuringElement(cv.MORPH_RECT,(7,7))

the main method of the program <b> 'update_image' </b>. in this method all the algorithm is being preformed. <br>
This method first takes a frame (aka image) and manipulates it in order to detect lines that are being treated the lanes. <br>
after that we calculate all the slopes of the lanes in order to know in which direction the jetRacer should go. <br>
Also this method detects a stop sign wherever it meets one and after a stop sign is being recognized than the jetRacer will stop completely and then proceed.



In [21]:
car.steering = -0.225
car.throttle = -0.62
image = camera.read() #gets the current frame.

import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

image_widget = ipywidgets.Image(format='jpeg')


display(image_widget)

camera.running = True

#this method gets each frame, detects the lanes and react according to this lanes. there is a stop sign detection too.
def update_image(change):
    image = change['new']
    # apply some gaussian blur to the image
    kenerl_size = (3, 3)
    gauss_image = cv.GaussianBlur(image, kenerl_size, 0)

    # here we convert to the HSV colorspace
    hsv_image = cv.cvtColor(gauss_image, cv.COLOR_BGR2HSV)

    # apply color threshold to the HSV image to get only black colors
    thres_1 = cv.inRange(hsv_image, lower_black, upper_black)

    # dilate the the threshold image
    thresh = cv.dilate(thres_1, rectKernel, iterations=1)

    # apply canny edge detection
    low_threshold = 200
    high_threshold = 400
    canny_edges = cv.Canny(gauss_image, low_threshold, high_threshold)
    # get a region of interest
    roi_image = region_of_interest(canny_edges)

    line_segments = detect_line_segments(roi_image)
    lane_lines = average_slope_intercept(image, line_segments)
    # overlay the line image on the main frame
    line_image = display_lines(image, lane_lines)

    length = len(lane_lines)
    
    #all the stop sign detection procedure
    img_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)  #makes the image gray.
    stop_data = cv.CascadeClassifier('stop_data.xml')
    found = stop_data.detectMultiScale(img_gray,  
                                        minSize =(20, 20))
    amount_found = len(found)
    
    #if stop sign detected.
    if amount_found != 0: 
        
        # There may be more than one 
        # sign in the image 
        for (x, y, width, height) in found: 

            # We draw a green rectangle around 
            # every recognized sign 
            cv.rectangle(line_image, (x, y),  
                            (x + height, y + width),  
                            (0, 255, 0), 5) 
        
        #slow down for a second and then make a full stop.
        car.throttle = -0.2
        sleep(1)
        car.throttle = 0
        
    
        
    
    #2 lanes detected
    if length == 2:
        left_x1, left_y1, left_x2, left_y2 = lane_lines[0][0]
        right_x1, right_y1, right_x2, right_y2 = lane_lines[1][0]
        slope_right = ((right_y2 - right_y1) / (right_x2 - right_x1))*(-1)
        slope_left = ((left_y2 - left_y1) / (left_x2 - left_x1))*(-1)
        
        
        #if the car is centered.
        if(slope_left + slope_right <= 0.5 and slope_left + slope_right >= -0.5):
            car.steering = -0.225
            car.throttle = -0.62
        
        #make the car centered (right).
        elif(slope_left + slope_right > 0.5 and slope_left + slope_right < 3):
            car.steering -= 0.02
            car.throttle += 0.015
        #make the car centered (left).   
        elif(slope_left + slope_right < -0.5 and slope_left + slope_right > -3):
            car.steering += 0.02
            car.throttle += 0.015
        #make the car centered (right, more aggressive).    
        elif(slope_left + slope_right > 3):
            car.steering -= 0.04
            car.throttle += 0.025
        #make the car centered (left, more aggressive).
        elif(slope_left + slope_right < -3):
            car.steering += 0.04
            car.throttle += 0.025
        #make sure the car isn't steering too much, because when there are two lanes, there is no need to make aggressive turns.  
        if car.steering < -0.625:
            car.steering = -0.625
        elif car.steering > 0.225:
            car.steering = 0.225
    
    #one lane detected
    elif length == 1:
        lane_x1, lane_y1, lane_x2, lane_y2 = lane_lines[0][0]
        slope = ((lane_y2 - lane_y1) / (lane_x2 - lane_x1))*(-1)
        
        #right lane is detected
        if slope <= 0 and slope >= -2:
            car.steering += 0.275
            car.throttle += 0.08
            
        #left lane is detected     
        elif slope >= 0 and slope <= 2:
            car.steering -= 0.325
            car.throttle += 0.15
            
        #right lane is detected - sharp angle.    
        elif slope < -2:
            car.steering += 0.375
            car.throttle += 0.1
            
        #left lane is detected - sharp angle. 
        elif slope > 2:
            car.steering -= 0.425
            car.throttle += 0.15
    #no lanes detected         
    else:
        car.throttle = -0.54
            
    #make sure the car's speed doesn't go above -0.575, because it may stopping the car(doesn't have enough power to move).
    if car.throttle > -0.575:
        car.throttle = -0.575
        
    
    image_widget.value = bgr8_to_jpeg(line_image) #display the image with the detected lines in the image widget
    
camera.observe(update_image, names='value') 

Image(value=b'', format='jpeg')

Image(value=b'', format='jpeg')

stop


This cell is the in charge of <b> stopping </b> the camera and then the jetRacer it self.

In [22]:
camera.unobserve(update_image, names='value') #stops the camera.
camera.running = False
car.throttle = 0 #stops the car.